In [ ]:
import abtem

# Performance, parallelization and GPUs

The cost of running simulations of transmission electron microscopy can range from seconds to months depending on factors such as sample size, number of probe positions and the hardware used to run the code. Here, we go through how optimize *abTEM* to get your simulations running as fast as possible.

## FFTs 

The [fast Fourier transform (FFT)](https://en.wikipedia.org/wiki/Fast_Fourier_transform) is the most important depending algorithm for running the multislice algorithm. Hence, the speed of your simulation depends on how fast your computer can run the FFT. This will depend on your hardware, the implementation of the FFT and the grid points of your simulation.  

### FFT Library (only on CPU)
*abTEM* provides two implementations of the FFT: Intels Math Kernel Library FFT ([`mkl_fft`](https://github.com/IntelPython/mkl_fft)) and the "Fastest Fourier Transform in the West" (FFTW) through the Python wrapper [`pyfftw`].(https://pyfftw.readthedocs.io/en/latest/). You can set the FFT library 

*abTEM* currently supports NVidia GPUs, hence  

abtem.config.set({})

## Dask

*abTEM* is parallelized using [Dask](https://dask.org/) {cite}`dask`. *abTEM* builds a *Dask* task graph for your simulation which is executed using one of the *Dask* schedulers.  

On a local system (a laptop or desktop), your is parallelized automatically using [threads](https://docs.dask.org/en/stable/scheduling.html#local-threads). Nonetheless, you may benefit from knowing about 

*abTEM* uses the 

as *abTEM


https://docs.dask.org/en/stable/scheduling.html

Parallel computation in abTEM whether it is running on a laptop or a High Performance Computing cluster is parallelized through . 

Some knowledge of Dask may be required to run abTEM effectively on large multi-CPU/GPU systems, however, if you are just running abTEM on your own computer, this tutorial gets you started. If you want a quick introduction to Dask, we recommend thishttps://www.youtube.com/watch?v=nnndxbr_Xq4&t=66s) short youtube video.



## GPUs

Almost every part . This is possible through

## Memory